In [1]:
%load_ext autoreload
%autoreload 2
import glob
import numpy as np
import pandas as pd
import numpy as np
from pathlib import Path
import json
from tqdm.notebook import tqdm, trange
from sglm.models import sglm
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf

import itertools

In [2]:
num_sft = (-20, 20)
# default_source_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\raw')
# default_output_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\interim')

default_source_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw')
default_output_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/interim')


output_json_name = Path('preproc_params.json')
basis_name = Path('_basis.csv')

In [3]:
y_col_lst_all = ['gACH', 'rDA', 'gDA', 'gGLU', 'rGLU',
                 'gACH_flx_drd', 'rDA_flx_drd', 'gDA_flx_drd',
                 'gACH_flx_cht', 'rDA_flx_cht', 'gDA_flx_cht',
                 'gACH_flx_drdcht', 'rDA_flx_drdcht', 'gDA_flx_drdcht',
                 'Ch5', 'Ch6',
                 'GP_1', 'GP_2', 'GP_5', 'GP_6',
                 'SGP_1', 'SGP_2', 'SGP_5', 'SGP_6']

In [4]:
params = {'sft': num_sft, 'y_cols_all': y_col_lst_all}
default_output_folder.mkdir(parents=True, exist_ok=True)
with open(str((default_output_folder / output_json_name).resolve()), 'w') as json_file:
    json.dump(params, json_file)

In [5]:
basis = pd.read_csv(str((default_source_folder / basis_name).resolve()))
basis.to_csv(str((default_output_folder / basis_name).resolve()))
source_folder = default_source_folder

for inx, conversion in basis.iterrows():
#     if 'source_folder' in conversion.index:
#         source_folder = Path(conversion['source_folder'])
#     else:
#         source_folder = default_source_folder
    
    figname = Path(conversion['figname'])
    groupid = Path(conversion['groupid'])
    signal_file = Path(conversion['signal_file'])
    table_file = Path(conversion['table_file'])
    rename_dict = eval(conversion['rename_dict'])
    signal_file_out = Path(conversion['signal_file_out'])
    table_file_out = Path(conversion['table_file_out'])
    
    signal_path = str((source_folder / signal_file).resolve())
    table_path = str((source_folder / table_file).resolve())
    
    out_folder = default_output_folder / figname / groupid
    out_folder.mkdir(parents=True, exist_ok=True)
    
    signal_path_out = default_output_folder / figname / groupid / signal_file_out
    table_path_out = default_output_folder / figname / groupid / table_file_out
    
    if signal_path_out.exists():
        raise OSError(f'Signal Path {signal_path_out} already exists!')
    if table_path_out.exists():
        raise OSError(f'Table Path {table_path_out} already exists!')
    
    signal_df = pd.read_csv(signal_path)
    table_df = pd.read_csv(table_path)
    
    # Check for multiple copies of the same sideIn
    eq = table_df['photometrySideInIndex'].dropna()
    eq = eq[eq != 0]
    if len(eq) != eq.nunique():
        print(len(eq))
        print(eq.nunique())
        display(eq)
        print(f'Error: Multiple side ins with the same timestamp detected for {signal_path}. Continuing...')
        continue
    
    print(signal_path)
    signal_df, table_df = gsd.generate_signal_df(signal_path,
                                            table_path,
                                            trial_bounds_before_center_in = num_sft[0],
                                            trial_bounds_after_side_out = num_sft[1],
                                            )
    
    signal_df = signal_df[signal_df['nTrial'] > 0].fillna(0)
    
    
    # Rename Columns
    signal_df = bf.rename_consistent_columns(signal_df)
    signal_df = signal_df.rename(rename_dict, axis=1)

    for y_col in y_col_lst_all:
        if y_col not in signal_df.columns:
            signal_df[y_col] = np.nan
            continue
    
    ## Set Full Trial Reward Flags
    signal_df['r_trial'] = (signal_df.groupby('nTrial')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
    signal_df['nr_trial'] = (signal_df.groupby('nTrial')['photometrySideInIndexnr'].transform(np.sum) > 0) * 1.0
#     signal_df['rnr_err_trial'] = (signal_df['r_trial'] + signal_df['nr_trial'] != 1.0) * 1.0

    ## Define Side Rewarded / Unrewarded Flags
    signal_df = bf.set_port_entry_exit_rewarded_unrewarded_indicators(signal_df)

    ## Define Side Agnostic Events
    signal_df = bf.define_side_agnostic_events(signal_df)
    
    # print('Percent of Data in ITI:', (df['nTrial'] == df['nEndTrial']).mean())

    signal_df['spnrOff'] = ((signal_df['spnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    signal_df['spxrOff'] = ((signal_df['spxr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)
    spnnrOff_a = ((signal_df['spnnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    spxnrOff_a = ((signal_df['spxnr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)

    # If we have something listed as a rewarded "off" side entry labeled in the table as a side exit... it means it was a fast "out-in".
    # The latter "in" should be considered an unrewarded side port "off" entry.
    dualism_exen = ((signal_df['spnrOff'] == 1)&(signal_df['photometrySideOutIndex'] == 1)).astype(int)

    # Unrewarded side port entries should be the combination of those simply identified by checking spnnr & the table labels +
    # the dualism defined immediately prior. Then those dualism examples should be remoed from the "off" rewarded entries.
    signal_df['spnnrOff'] = spnnrOff_a + dualism_exen
    signal_df['spnrOff'] = signal_df['spnrOff'] - dualism_exen
    signal_df['spxnrOff'] = spxnrOff_a
    
    signal_df['slOff'] = signal_df['sl'] * signal_df['nr_trial']
    signal_df['slOn'] = signal_df['sl'] - signal_df['slOff']
    
    signal_df['cpnOff'] = ((signal_df['cpn'] == 1)&(signal_df['photometryCenterInIndex'] != 1)).astype(int)
    signal_df['cpxOff'] = ((signal_df['cpx'] == 1)&(signal_df['photometryCenterOutIndex'] != 1)).astype(int)
    
    if signal_path_out:
        signal_df.to_csv(signal_path_out, index_label='index')
    if table_path_out:
        table_df.to_csv(table_path_out, index_label='index')
    
    assert np.max(signal_df['photometrySideInIndexr'] + signal_df['photometrySideInIndexnr']) == 1.0

/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw/GLM_SIGNALS_S1299_02022022.txt
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1
/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw/GLM_SIGNALS_S1299_02072022.txt
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1
/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw/GLM_SIGNALS_S1300_02022022.txt
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1
/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw/GLM_SIGNALS_S1300_02072022.txt
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1
/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw/GLM_SIGNALS_S1301_01312022.txt
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1
/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw/GLM_SIGNALS_S1301_02022022.txt
# of iterations 0 — Final max amount of duplicated Center O

In [6]:
# signal_df[['photometrySideInIndex', 'photometrySideInIndexr', 'photometrySideInIndexnr',
#            'r_trial', 'nr_trial', 'rnr_err_trial', 'nTrial', 'nEndTrial']].loc[signal_df['rnr_err_trial'] == 1.0]


In [7]:
# basis = pd.read_csv(str((default_source_folder / f'_basis.csv').resolve()))
# source_folder = default_source_folder

# for inx, conversion in basis.iterrows():
#     figname = Path(conversion['figname'])
#     groupid = Path(conversion['groupid'])
#     signal_file = Path(conversion['signal_file'])
#     table_file = Path(conversion['table_file'])
#     rename_dict = eval(conversion['rename_dict'])
#     signal_file_out = Path(conversion['signal_file_out'])
#     table_file_out = Path(conversion['table_file_out'])
    
    
#     signal_path = str((source_folder / signal_file).resolve())
#     table_path = str((source_folder / table_file).resolve())
    
    
#     signal_path_out = default_output_folder / figname / groupid / signal_file_out
#     table_path_out = default_output_folder / figname / groupid / table_file_out
        
#     signal_df_out = pd.read_csv(signal_path_out)#.drop('rnr_err_trial',axis=1)
#     table_df_out = pd.read_csv(table_path_out)
#     old_signal_df = pd.read_csv(str(Path((r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\interim-old') / signal_file_out).resolve()))
    
#     assert np.all(old_signal_df.fillna(-1) == signal_df_out.fillna(-1))
    